# REDE DE CITAÇÕES - SCOPUS - DOENÇAS INFECCIOSAS

In [1]:
# CARREGANDO BIBLIOTECAS
import warnings
warnings.simplefilter(action='ignore', category=DeprecationWarning)
import OpenSSL.SSL
import getpass

import pybliometrics
import pybliometrics.scopus as sc

from selenium import webdriver
from bs4 import BeautifulSoup

from pymongo import MongoClient
import pandas as pd
import pickle

from math import *

import logging
import os
import shutil
import send2trash
import time
import itertools
import gc 
from tqdm import tqdm_notebook as tqdm

import configparser as cf
import importlib

path_atual = os.getcwd()
print(path_atual)
path_download = '../../Downloads/'
path_scopus = '../../../Walter/.scopus'

C:\Users\Walter\Documents\WOS_CitationNetworks


O **pybliometrics** (anteriormente **scopus**) usa um arquivo de configuração `~/.scopus/config.ini` para salvar credenciais e nomes de diretório da pasta que armazena arquivos em cache.

Para alterar sua chave ou alterar diretórios para arquivos em cache, edite esse arquivo manualmente e importe pybliometrics novamente.

Se o arquivo de configuração não existir, o pybliometrics emitirá um aviso. Para gerar o arquivo de configuração, faça o seguinte comando `pybliometrics.utils.create_config()` logo após a importação.


A **pybliometrics** solicita suas credenciais. A maioria dos usuários só precisa fornecer a chave da API e pressionar Enter no segundo prompt. 

In [2]:
# CONFIGURAÇÕES
#sc.utils.create_config()

In [3]:
# CATEGORIA PESQUISADA
category = 'INFEC-DISEASES'

## 1. No WoS, escolher uma área de pubicação e obter a listagem de periódicos.

Usando o Web of Science, obter uma listagem de todos os periódicos publicados na categoria escolhida (**MEDICINE**). A fim de filtrar os resultados, foi escolhida uma subárea: **INFECTIOUS DISEASES**.

* Em `Advanced Search` do WoS, pesquisar: `WC=(MEDICINE)` e selecionar a opção de apenas artigos. São encontrados 2.633.199 artigos. 


* Em seguida, selecionar o filtro de categoria do WoS, `INFECTIOUS DISEASES`. Agora, são eoncontrados 19.353 artigos. 



* No campo de filtros, tem a opção `Source Titles`. Ir em `more options / values...`. Com a página atualizada, ir em ` Analyze results`.


* Novamente, escolher a opção `Source Titles`. 


* No final da página, selecionar a opção `All data rows (up to 100,000)` e realizar o download. 

![](figuras/journalsVis_INFEC-DISEASES.jpg)

In [4]:
file_journals = pd.read_csv('journals/SourceTitles_' + category + '.txt', sep='\t',index_col=False)
file_journals.index += 1 
file_journals = file_journals[:-1]

In [5]:
file_journals

,Source Titles,records,% of 19353
1,PLOS NEGLECTED TROPICAL DISEASES,6183.0,31.949
2,MALARIA JOURNAL,5143.0,26.575
3,BULLETIN DE LA SOCIETE DE PATHOLOGIE EXOTIQUE,2286.0,11.812
4,LEPROSY REVIEW,1616.0,8.350
5,SOUTHEAST ASIAN JOURNAL OF TROPICAL MEDICINE A...,1522.0,7.864
6,JOURNAL OF TRAVEL MEDICINE,1248.0,6.449
7,ADVANCES IN EXPERIMENTAL MEDICINE AND BIOLOGY,728.0,3.762
8,JOURNAL OF VECTOR BORNE DISEASES,498.0,2.573
9,STREPTOCOCCI AND THE HOST,243.0,1.256
10,ANNALS OF THE NEW YORK ACADEMY OF SCIENCES,57.0,0.295


In [6]:
SourceTitles = list(file_journals['Source Titles'])

In [7]:
file_years = pd.read_csv('journals/PublicationYears_' + category + '.txt', sep='\t',index_col=False)
file_years.index += 1 
file_years = file_years[:-2]

In [8]:
file_years.head()

,Publication Years,records,% of 19353
1,2019,434.0,2.243
2,2018,1474.0,7.616
3,2017,1657.0,8.562
4,2016,1711.0,8.841
5,2015,1570.0,8.112


In [9]:
file_years.tail()

,Publication Years,records,% of 19353
41,1979,82.0,0.424
42,1978,92.0,0.475
43,1977,110.0,0.568
44,1976,68.0,0.351
45,1974,23.0,0.119


In [10]:
PublicationYears = list(file_years['Publication Years'])

## 2. Usando a API do Scopus, pesquisar os artigos publicados por cada periódico.

Usando a função [**ScopusSearch**](https://scopus.readthedocs.io/en/stable/examples/ScopusSearch.html), buscamos pelo nome de cada periódico (`SourceTitles`) em um intervalo de tempo pré-definido de 1974 a 2018. 

* O nome do periódico deve ser colocado entre aspas duplas para ser pesquisado como uma string, caso contrário será pesquisado como uma interseção das palavras que estão contidas no nome. 

` documents = sc.ScopusSearch('SRCTITLE ("' + str(journal) + '")  AND  PUBDATETXT (' + str(year) + ') AND DOCTYPE(ar) AND SRCTYPE (j)', refresh=True, apiKey=keys[k])`

`documents = sc.ScopusSearch('SRCTITLE ( "PLOS NEGLECTED TROPICAL DISEASES" )  AND  PUBDATETXT (2018) AND DOCTYPE(ar) AND SRCTYPE (j)', refresh=True, apiKey=keys[k])`

Como podemo gerar apenas 10 chaves de uma só vez, todas foram armazenadas em uma lista para ser usada durante o processo de extração. Caso o valor seja excedido, a extração irá parar e novas chaves derão ser geradas [manualmente](https://dev.elsevier.com/apikey/manage) e substituídas na lista `keys`. 

In [11]:
def createKeys(password,email,op=1):
    # op=1: gera as 10 primeiras chaves (ainda não tenm nenhuma chave gerada)
    # op=2: gera 10 novas chaves (deleta as 10 já existentes e depois faz a geração das chaves)
    
    driver = webdriver.Chrome(executable_path = r'/Users/Walter/Documents/chromedriver')

    scopus_api_link = 'https://dev.elsevier.com/apikey/manage'
    driver.get(scopus_api_link)

    driver.find_element_by_xpath('//*[@id="bdd-email"]').send_keys(email)
    driver.find_element_by_xpath('//*[@id="bdd-elsPrimaryBtn"]').click()
    driver.find_element_by_xpath('//*[@id="bdd-password"]').send_keys(password)
    driver.find_element_by_xpath('//*[@id="bdd-elsPrimaryBtn"]').click()
    
    if op==2:
        # deletar todas as dez chaves existentes
        for key in range(1,11):
            time.sleep(5)
            driver.find_element_by_xpath('/html/body/div[1]/div[4]/div[2]/div[1]/table/tbody/tr[1]/td[4]/a').click()
            driver.find_element_by_xpath('//*[@id="delregister"]').click()
            time.sleep(5)
            driver.find_element_by_xpath('//*[@id="deleteModal"]/div/div/div[3]/button[1]').click()
    
    if op==1 or op==2:
        # criar todas as dez chaves permitidas
        for key in range(1,11):
            time.sleep(5)
            driver.find_element_by_xpath('/html/body/div[1]/div[4]/div[2]/h2/span/a').click()
            driver.find_element_by_xpath('//*[@id="projectName"]').send_keys('scopus'+str(key))
            driver.find_element_by_xpath('//*[@id="websiteURL"]').send_keys('https://github.com/anacwagner/')
            driver.find_element_by_xpath('//*[@id="apicheckbox"]/div/label').click()  
            driver.find_element_by_xpath('//*[@id="tdmcheckbox"]/div/label').click()
            driver.find_element_by_xpath('//*[@id="register"]').click()   
        
    
    soup = BeautifulSoup(driver.page_source,'lxml')
    findAll = soup.findAll('a')
    
    keys = []
    for i in range(10,20):
        keys.append(findAll[i].text)
        
    driver.quit()
    
    return keys

In [12]:
password = getpass.getpass('Password:')
email = 'acwgdb@gmail.com'

Password:········


In [13]:
keys = createKeys(password,email,2) 
keys

['7e7f67a159fb90afa033290f852f4131',
 'd4a42bfd7f407e2b6c13b09b21dcbc14',
 '7be63008d9cefda790203b1c27e04569',
 'd82f760c53ffa7761301e565b600d8a2',
 '95f8268d60e2a4d324da7d234e6e043d',
 '421526296215ebcc3b3bcc2f28a3a4b6',
 '03f58bf9c3361b6337269d3723ff1c4c',
 '80f697f62db1aeb9490a66b7b408d220',
 '46ef9059698572beb9a55b76157fca29',
 '344bdf6291d72b713fb9b440620bb3cf']

Sempre que gerarmos novas chaves, devemos modificá-la no arquivo de configurações.

In [14]:
def changeKeys(k,keys):
    config = cf.ConfigParser()
    config.read(path_scopus + '/config.ini')
    config.set("Authentication", "apikey", keys[k])
    with open(path_scopus + '/config.ini', "w+") as configfile:
        config.write(configfile)
    print('Nova Chave: ' + config.get("Authentication", "apikey"))

In [15]:
changeKeys(0,keys)

Nova Chave: 7e7f67a159fb90afa033290f852f4131


Segue um exemplo da pesquisa que vamos realizar.

In [16]:
documents = sc.ScopusSearch('SRCTITLE ("' + str(SourceTitles[0]) + '")  AND  PUBDATETXT (' + str(2018) + ') AND DOCTYPE(ar) AND SRCTYPE ( j )', refresh=True, apiKey=keys[0])
print('Total de resultados encontrados: ' + str(documents.get_results_size()) + ' artigos.')
df_documents = pd.DataFrame(pd.DataFrame(documents.results))
df_documents.index += 1 
df_documents.head(2)

Total de resultados encontrados: 823 artigos.


,eid,doi,pii,pubmed_id,title,subtype,creator,afid,affilname,affiliation_city,...,issueIdentifier,article_number,pageRange,description,authkeywords,citedby_count,openaccess,fund_acr,fund_no,fund_sponsor
1,2-s2.0-85059543982,10.1371/journal.pntd.0007016,None,30576335,General contextual effects on neglected tropic...,ar,de Glanville W.,60070343;60057834;60027272;60026158;60025225;6...,Kenya Medical Research Institute;International...,Nairobi;Nairobi;Edinburgh;Midlothian;Southampt...,...,12,e0007016,None,© 2018 de Glanville et al. http://creativecomm...,None,0,1,None,085308,Biotechnology and Biological Sciences Research...
2,2-s2.0-85059538668,10.1371/journal.pntd.0007063,None,30586355,Evaluation of the analytical and diagnostic pe...,ar,Ramírez J.,60065589;60040016;60031331,Universidad del Rosario;Instituto Nacional de ...,Bogota;Bogota;London,...,12,e0007063,None,© 2018 Ramírez et al. http://creativecommons.o...,None,2,1,None,undefined,None


O resultado da pesquisa sobre `documents` nos permite ter acesso às seguintes informações:

Index(['eid', 'doi', 'pii', 'pubmed_id', 'title', 'subtype', 'creator', 'afid',
       'affilname', 'affiliation_city', 'affiliation_country', 'author_count',
       'author_names', 'author_ids', 'author_afids', 'coverDate',
       'coverDisplayDate', 'publicationName', 'issn', 'source_id', 'eIssn',
       'aggregationType', 'volume', 'issueIdentifier', 'article_number',
       'pageRange', 'description', 'authkeywords', 'citedby_count',
       'openaccess', 'fund_acr', 'fund_no', 'fund_sponsor'],
      dtype='object')

### Mensagens de Erro
Uma exceção é gerada quando o status do download não está ok. Isso evita que informações com falha (ou seja, o status e a mensagem do erro) sejam salvas como arquivo em cache. Essas exceções são da classe base ScopusException.

Algumas delas, são:

 
* `pybliometrics.scopus.exception.Scopus404Error`: **NOT FOUND**
A entidade que você está procurando não existe. Verifique se o seu identificador ainda está apontando para o item que você está procurando.


* `pybliometrics.scopus.exception.Scopus429Error`: **QUOTA EXCEEDED**
O limite semanal de 5000 solicitações da chave de API fornecida (para visualizações padrão) está esgotado. Aguarde uma semana ou altere a chave em `~/.scopus /config.ini.`.


* `pybliometrics.scopus.exception.Scopus500Error`: **INTERNAL SERVER ERROR**
Formalmente, o servidor não responde, por vários motivos. Um motivo comum nas pesquisas é que você usa um nome de campo que não existe. Verifique se sua consulta funciona na Pesquisa avançada.

In [17]:
def allArticlesSource(journalsSourceTitles, PublicationYears):
    
    source_eids = []
    
    i = len(source_eids) + 1
    k = 0 # controle da chave da API
    
    for journal in journalsSourceTitles[i-1:]:
        print('\n ' + str(i) + '. Início da extração dos artigos do periódico \n' + journal)
        
        eids = []
        
        for year in range(int(PublicationYears[-1]),2019):
            
            resultado = False
            
            while resultado == False:
                try: 
                    documents = sc.ScopusSearch('SRCTITLE ("' + str(journal) + '")  AND  PUBDATETXT (' + str(year) + ') AND DOCTYPE(ar) AND  SRCTYPE (j)')
                    eids.append(documents.get_eids())
                    resultado = True
                    
                except sc.exception.Scopus404Error:  #NOT FOUND
                    resultado = True
                except sc.exception.Scopus429Error : #QUOTA EXCEEDED
                    k = k + 1
                    if k == 11:
                        keys = createKeys(password,email)
                        k = 0
                        changeKeys(k,keys)
                    else:
                        changeKeys(k,keys)
                except sc.exception.Scopus500Error:  #INTERNAL SERVER ERROR
                    time.sleep(14)

        source_eids.append(eids)
        i = i + 1

        with open('pickles/' + category + ' _eids.pickle', 'wb') as f:
            pickle.dump([source_eids], f)
        
        #os.remove(path_scopus + '/scopus_search/COMPLETE/' + eid)
        gc.collect()

In [18]:
#allArticlesSource(SourceTitles, PublicationYears)

In [19]:
with open('pickles/' + category + '_eids.pickle', 'rb') as f:
    source_eids = pickle.load(f)

A variável `source_eids` é uma lista de listas que armazena os códigos de identificação dos artigos publicados por cada periódico em cada ano. Cada ano é uma lista dentro da lista das publicações do periódico correpondente.

In [20]:
len(source_eids[0])

49

In [21]:
articlesSource = list(itertools.chain.from_iterable(source_eids))
articlesSource = list(itertools.chain.from_iterable(articlesSource))
articlesSource = list(set(list(itertools.chain.from_iterable(articlesSource))))
print('Total de artigos da base: ' + str(len(articlesSource)))

Total de artigos da base: 41247


## 3. Usando a API do Scopus, extrair e armazenar as informações de cada artigo.

As informações extraídas são armazenadas em dois diconários, um contendo as informações dos artigos (**info_articles**) e outro contendo as informações dos periódicos (**info_journals**). Essas estruturas foram pensadas para serem utilizadas no **PyMongo**!

In [22]:
def keyNumber():
    # Qual a chave que está sendo utilizada
    config = cf.ConfigParser()
    config.read(path_scopus + '/config.ini')
    #print(config.get("Authentication", "apikey"))
    k = keys.index(str(config.get("Authentication", "apikey")))
    return k

In [23]:
def articlesInfo(articles_eids,keys):
    
    # Qual a chave que está sendo utilizada
    k = keyNumber()
    
    info_journals = {}
    info_articles = {}
    n = len(articles_eids)
    
    for i in tqdm(range(n)):
        eid = articles_eids[i]
        
        id_article = len(info_articles) + 1
        #print('Início da extração das informações do artigo ' + str(id_article))

        resultado = False 
        while resultado == False:
            try: 
                infos = sc.AbstractRetrieval(eid , view='FULL', refresh=True)
                resultado = True
                delete = True
            except sc.exception.Scopus404Error:  #NOT FOUND
                resultado = True
            except sc.exception.Scopus429Error : #QUOTA EXCEEDED
                k = k + 1
                if k == 11:
                    keys = createKeys(password,email)
                    k = 0
                    changeKeys(k,keys)
                else:
                    changeKeys(k,keys)
            except sc.exception.Scopus500Error:  #INTERNAL SERVER ERROR
                time.sleep(14)

        if infos.issn != None:

            aux_issn = infos.issn.split(' ')
            m=0
            n=1
            issn = aux_issn[0]

            if len(aux_issn) == 1: 
                n = 0

            if info_journals.get(aux_issn[m]) == None and info_journals.get(aux_issn[n]) == None:

                id_journal = len(info_journals) + 1
                info_journals[issn] = {"ID_J": id_journal,
                                       "TITLE_J": infos.publicationName}

            elif info_journals.get(aux_issn[0]) != None:
                id_journal = info_journals[issn]['ID_J']

            else:
                issn = aux_issn[1]
                id_journal = info_journals[issn]['ID_J']

        else: 
            issn = infos.publicationName
            if info_journals.get(issn) == None:
                id_journal = len(info_journals) + 1
                info_journals[issn] = {"ID_J": id_journal,
                                       "TITLE_J": infos.publicationName}
            else:
                id_journal = info_journals[issn]['ID_J']

        refs = infos.references
        refs_eids = []
        if refs != None:
            for r in range(len(refs)):
                refs_eids.append(refs[r][1])

        authors = {}
        if infos.authors != None:
            for a in range(len(infos.authors)):
                authors['author_' + str(a)] ={'auid': infos.authors[a][0],
                                              'indexed_name': infos.authors[a][1],
                                              'affiliation': infos.authors[a][-1]}

        try: 
            cited = infos.citedby_count
        except KeyError:
            cited = 'NaN'

        info_articles[eid] = {"ID_A": id_article,
                              "TITLE_A": infos.title,
                              "DOI": infos.doi,
                              "YEAR": infos.coverDate,
                              "AUTHORS": authors,
                              "ISSN": issn,
                              "ID_J": id_journal,
                              "TITLE_J": infos.publicationName,
                              "REFS": refs_eids,
                              "CITED_BY_COUNT": cited}

        gc.collect()
        
        try:
            os.remove(path_scopus + '/abstract_retrieval/FULL/' + eid)
        except FileNotFoundError:
            continue

        with open('pickles/' + category + '_INFOS.pickle', 'wb') as f:
                pickle.dump([info_articles,info_journals], f)
        pass
    
    with open('pickles/' + category + '_INFOS.pickle', 'wb') as f:
        pickle.dump([info_articles,info_journals], f)

In [24]:
articlesInfo(articlesSource,keys)

Nova Chave: 1518a4b12db5a243db8fd753960959b8
Nova Chave: 693fbc17524c91d69deaddb6acd19981
Nova Chave: 3d1e2714d92d51b456fe658b295890aa
Nova Chave: d9a0c6198b679b68d3a98d2e881ec67e



## 4. Usando a API do Scopus, extrair e armazenar as informações das referências de cada artigo.

In [25]:
with open('pickles/' + category + '_INFOS.pickle', 'rb') as f:
    [info_articles, info_journals] = pickle.load(f)

In [26]:
print('Total de Periódicos: ' + str(len(info_journals)))

Total de Periódicos: 22


In [27]:
for i in range(len(SourceTitles)):
    if list(itertools.chain.from_iterable(source_eids[0][i])) == []:
        print('Não foram coletadas informações do periódico:', str(i), SourceTitles[i])

Não foram coletadas informações do periódico: 8 STREPTOCOCCI AND THE HOST
Não foram coletadas informações do periódico: 10 IMPACT OF ECOLOGICAL CHANGES ON TROPICAL ANIMAL HEALTH AND DISEASE CONTROL
Não foram coletadas informações do periódico: 11 MODULATION OF THE IMMUNE RESPONSE TO VACCINE ANTIGENS
Não foram coletadas informações do periódico: 13 MECHANISMS IN THE PATHOGENESIS OF ENTERIC DISEASES 2
Não foram coletadas informações do periódico: 14 NEUROIMMUNE CIRCUITS DRUGS OF ABUSE AND INFECTIOUS DISEASES
Não foram coletadas informações do periódico: 15 HOT TOPICS IN INFECTION AND IMMUNITY IN CHILDREN
Não foram coletadas informações do periódico: 16 TROPICAL DISEASES FROM MOLECULE TO BEDSIDE
Não foram coletadas informações do periódico: 17 DENGUE AND ZIKA CONTROL AND ANTIVIRAL TREATMENT STRATEGIES
Não foram coletadas informações do periódico: 18 POLYOMAVIRUSES AND HUMAN DISEASES
Não foram coletadas informações do periódico: 19 ANTIVIRAL CHEMOTHERAPY 5
Não foram coletadas informações d

In [28]:
print('Total de Artigos: ' + str(len(info_articles)))

Total de Artigos: 41247


In [29]:
info_journals['14752875']

{'ID_J': 1, 'TITLE_J': 'Malaria Journal'}

In [30]:
info_articles['2-s2.0-84858304335']

{'ID_A': 1,
 'TITLE_A': 'Modelling the dynamics of Plasmodium falciparum histidine-rich protein 2 in human malaria to better understand malaria rapid diagnostic test performance',
 'DOI': '10.1186/1475-2875-11-74',
 'YEAR': '2012-03-20',
 'AUTHORS': {'author_0': {'auid': '57202807585',
   'indexed_name': 'Marquart L.',
   'affiliation': ['60025029']},
  'author_1': {'auid': '36670828200',
   'indexed_name': 'Butterworth A.',
   'affiliation': ['60025029', '60031004']},
  'author_2': {'auid': '55649177700',
   'indexed_name': 'McCarthy J.S.',
   'affiliation': ['60025029', '60031004']},
  'author_3': {'auid': '6701868710',
   'indexed_name': 'Gatton M.L.',
   'affiliation': ['60093234']}},
 'ISSN': '14752875',
 'ID_J': 1,
 'TITLE_J': 'Malaria Journal',
 'REFS': ['31444454856',
  '0030751388',
  '24044463572',
  '54349108204',
  '70350401148',
  '0035295408',
  '33846043474',
  '21044447731',
  '0003950237',
  '80051922283',
  '79951944951',
  '0032765519',
  '0032791514',
  '79960658403

In [31]:
info_articles['2-s2.0-84858304335']['REFS']

['31444454856',
 '0030751388',
 '24044463572',
 '54349108204',
 '70350401148',
 '0035295408',
 '33846043474',
 '21044447731',
 '0003950237',
 '80051922283',
 '79951944951',
 '0032765519',
 '0032791514',
 '79960658403',
 '79955122444',
 '79953896041',
 '77956466814',
 '56349172651',
 '68949151495',
 '77955558696',
 '84861338996']

Como podemos perceber, a estrtura da codifcação dos artigos da base está diferente das referências citadas pelo mesmo (sequência apenas numérica). A codificação dos artigos da base possuem a mesma sequência numérica, porém com o seguinte prefixo: **2-s2.0-**. Se não deixarmos padronizadas, como tais sequências são as chaves do dicionário com as informações de todos os artigos, se algum artigo da base aparecer nas referências de outro artigo, ele será inserido novamente no dicionário com outra chave.

In [34]:
def refsInfo(source_eids, info_journals, info_articles, keys):
    
    # Qual a chave que está sendo utilizada
    k = keyNumber()

    art = 0
    n = len(source_eids)
    
    ref_e = []
    for i in tqdm(range(n)):
        eid = source_eids[i]
        REFS = info_articles[eid]['REFS']
        art = art +  1
        #print('Início da extração das informações das citações do artigo ' + str(art))
        cit = 0
        
        if len(REFS) != 0:
            for ref in REFS:
                cit = cit + 1
                ref = '2-s2.0-' + ref
                if info_articles.get(ref) == None:
                    
                    id_article = len(info_articles) + 1
                    
                    resultado = False 
                    while resultado == False:
                        try: 
                            infos = sc.AbstractRetrieval(ref , view='FULL', refresh=True)
                            resultado = True
                            ref_info = True
                        except sc.exception.Scopus404Error:  #NOT FOUND
                            resultado = True
                            ref_info = False
                        except sc.exception.Scopus429Error : #QUOTA EXCEEDED
                            k = k + 1
                            if k == 11:
                                keys = createKeys(password,email)
                                k = 0
                                changeKeys(k,keys)
                            else:
                                changeKeys(k,keys)
                        except sc.exception.Scopus500Error:  #INTERNAL SERVER ERROR
                            time.sleep(14)
                    
                    if not(ref_info): 
                        ref_e.append(ref)
                        continue
                    
                    if infos.issn != None:
                        aux_issn = infos.issn.split(' ')
                        m=0
                        n=1
                        issn = aux_issn[0]

                        if len(aux_issn) == 1: 
                            n = 0
                        
                        if info_journals.get(aux_issn[m]) == None and info_journals.get(aux_issn[n]) == None:

                            id_journal = len(info_journals) + 1
                            info_journals[issn] = {"ID_J": id_journal,
                                                   "TITLE_J": infos.publicationName}

                        elif info_journals.get(aux_issn[0]) != None:
                            id_journal = info_journals[issn]['ID_J']

                        else:
                            issn = aux_issn[1]
                            id_journal = info_journals[issn]['ID_J']

                    else: 
                        issn = infos.publicationName
                        if info_journals.get(issn) == None:
                            id_journal = len(info_journals) + 1
                            info_journals[issn] = {"ID_J": id_journal,
                                                   "TITLE_J": infos.publicationName}
                        else:
                            id_journal = info_journals[issn]['ID_J']


                    refs = infos.references
                    refs_eids = []
                    if refs != None:
                        for r in range(len(refs)):
                            refs_eids.append(refs[r][1])

                    authors = {}
                    if infos.authors != None:
                        for a in range(len(infos.authors)):
                            authors['author_' + str(a)] ={'auid': infos.authors[a][0],
                                                          'indexed_name': infos.authors[a][1],
                                                          'affiliation': infos.authors[a][-1]}

                    try: 
                        cited = infos.citedby_count
                    except KeyError:
                        cited = 'NaN'

                    info_articles[ref] = {"ID_A": id_article,
                                          "TITLE_A": infos.title,
                                          "DOI": infos.doi,
                                          "YEAR": infos.coverDate,
                                          "AUTHORS": authors,
                                          "ISSN": issn,
                                          "ID_J": id_journal,
                                          "TITLE_J": infos.publicationName,
                                          "REFS": refs_eids,
                                          "CITED_BY_COUNT": cited}

                    gc.collect()
                    
                    try:
                        os.remove(path_scopus + '/abstract_retrieval/FULL/' + ref)
                    except FileNotFoundError:
                        continue      

            with open('pickles/' + category + '_REFS.pickle', 'wb') as f:
                pickle.dump([info_articles,info_journals, ref_e], f)
        pass

    with open('pickles/' + category + '_REFS.pickle', 'wb') as f:
        pickle.dump([info_articles,info_journals, ref_e], f)

In [ ]:
refsInfo(articlesSource, info_journals, info_articles, keys)

Nova Chave: d4a42bfd7f407e2b6c13b09b21dcbc14


In [ ]:
with open('pickles/' + category + '_REFS.pickle', 'rb') as f:
    [info_articles, info_journals, ref_e] = pickle.load(f)

In [ ]:
print("Total de Artigos: " + str(len(info_articles)))
print("Total de Periódicos: " + str(len(info_journals)))